In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=gemini_api_key)
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000028F9F4737A0>, default_metadata=())

In [25]:
while True:
    question = input("type your question. if you want to quit the chat write quit")
    if question != "quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hi there! How can I help you today?
goodbye take care yourself


In [26]:
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [27]:
store= {}

In [28]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [29]:
config = {"configurable": {"session_id": "firstChat"}}

In [30]:
model_with_memory = RunnableWithMessageHistory(llm,get_session_history)

In [31]:
model_with_memory.invoke(("Hi! I'm Neal Patel"),config=config).content

"Hi Neal Patel! It's nice to meet you. How can I help you today?"

In [32]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Neal Patel.'

RAG WITH LCEL

In [33]:
from langchain_community.document_loaders import TextLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAIEmbeddings
loader = DirectoryLoader('../data', glob="./*.txt",loader_cls=TextLoader)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
   )
documents = text_splitter.split_documents(documents=docs)
vectordb = Chroma.from_documents(documents,GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=gemini_api_key))
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000028F9F49E900>, search_kwargs={})

In [34]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [35]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm 
    | StrOutputParser()
)

In [36]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided context:

1.  Llama 3 has an 8B parameter version.
2.  Llama 3 was released in April 2024.


Let's Start with Tools and Agents

In [37]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=200)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
tool.name

'wikipedia'

In [38]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [39]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain


In [40]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain'

youtube search tool

In [43]:
from langchain_community.tools import YouTubeSearchTool
tool2= YouTubeSearchTool()
tool2.name

'youtube_search'

In [45]:
tool2.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [54]:
tool2.run("sunny savita")

"['https://www.youtube.com/watch?v=ktjJAxaX8rc&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=cydntJObPLI&pp=ygUMc3Vubnkgc2F2aXRh']"

In [49]:
# from langchain_community.tools.tavily_search import TavilySearchResults
# tool3 = TavilySearchResults()
# tool3.invoke({"query": "What happened in the latest burning man floods"})

In [50]:
from langchain.agents import AgentType, load_tools, initialize_agent

tool = load_tools(["wikipedia"],llm=llm)

agent = initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\Parth\AppData\Local\Temp\ipykernel_15128\2503053091.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [53]:
agent.run("who is known as God of cricket ?")



> Entering new AgentExecutor chain...
I need to find out who is known as the God of Cricket. I can use Wikipedia to search for this information.
Action: wikipedia
Action Input: God of cricket
Observation: Page: Sachin Tendulkar
Summary: Sachin Ramesh Tendulkar ( ; Marathi: [sətɕin t̪eɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest cricketers of all time, and is the holder of several world records, including being the all-time highest run-scorer in both ODI and Test cricket, receiving the most player of the match awards in international cricket, and being the only batsman to score 100 international centuries. Tendulkar was a Member of Parliament, Rajya Sabha by presidential nomination from 2012 to 2018. 
Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumb

'Sachin Tendulkar is widely known as the God of Cricket.'